Importing libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
#from skimpy import skim
import datetime
import pyodbc

import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

In [2]:
# import requests
# import pandas as pd

# # URL de la API del Banco Mundial para PIB a precios corrientes (indicador: NY.GDP.MKTP.CD)
# url = "http://api.worldbank.org/v2/country/all/indicator/NY.GDP.MKTP.CD"
# params = {
#     "date": "1960:2024",  # Rango de años
#     "format": "json",     # Formato de respuesta
#     "per_page": "30000"    # Número de registros por página (ajusta según tus necesidades)
# }

# # Realiza la solicitud a la API
# response = requests.get(url, params=params)
# data = response.json()

# # Extrae los datos de interés
# gdp_data = []
# for entry in data[1]:
#     country = entry['country']['value']
#     year = entry['date']
#     gdp = entry['value']
#     gdp_data.append([country, year, gdp])

# # Convierte los datos en un DataFrame de pandas
# GDPdf = pd.DataFrame(gdp_data, columns=["Country", "Year", "GDP_Current_USD"])
# GDPdf

# # Guarda los datos en un archivo CSV si lo deseas
# #df2.to_csv("world_gdp_1960_2021.csv", index=False)


In [3]:
# import requests
# import pandas as pd

# # URL de la API del Banco Mundial para PIB a precios corrientes (indicador: NY.GDP.MKTP.CD)
# url = "http://api.worldbank.org/v2/country/all/indicator/SP.POP.1564.TO"
# params = {
#     "date": "1960:2023",  # Rango de años
#     "format": "json",     # Formato de respuesta
#     "per_page": "30000"    # Número de registros por página (ajusta según tus necesidades)
# }

# # Realiza la solicitud a la API
# response = requests.get(url, params=params)
# data = response.json()

# # Extrae los datos de interés
# pob_data = []
# for entry in data[1]:
#     country = entry['country']['value']
#     year = entry['date']
#     Population = entry['value']
#     pob_data.append([country, year, Population])

# # Convierte los datos en un DataFrame de pandas
# df3 = pd.DataFrame(pob_data, columns=["Country", "Year", "Population ages 15-64"])
# df3

In [4]:
# import requests
# import pandas as pd

# # URL de la API del Banco Mundial para PIB a precios corrientes (indicador: NY.GDP.MKTP.CD)
# url = "http://api.worldbank.org/v2/country/all/indicator/FP.CPI.TOTL.ZG"
# params = {
#     "date": "1960:2023",  # Rango de años
#     "format": "json",     # Formato de respuesta
#     "per_page": "30000"    # Número de registros por página (ajusta según tus necesidades)
# }

# # Realiza la solicitud a la API
# response = requests.get(url, params=params)
# data = response.json()

# # Extrae los datos de interés
# Infl_data = []
# for entry in data[1]:
#     country = entry['country']['value']
#     year = entry['date']
#     Infl = entry['value']
#     Infl_data.append([country, year, Infl])

# # Convierte los datos en un DataFrame de pandas
# df4 = pd.DataFrame(Infl_data, columns=["Country", "Year", "Inflation"])
# df4

In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime
import requests
import warnings
import os # Importar el módulo os para manejar rutas de archivos

warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

# Lista de indicadores del Banco Mundial y los nombres de las hojas que quieres para cada uno
INDICATORS = [
    ("NY.GDP.MKTP.CD", "GDP_Current_USD", "GDP_Current_USD"),  # PIB a precios corrientes
    ("SP.POP.1564.TO", "Population_ages_15-64", "Population_15-64"),  # Población entre 15-64 años
    ("FP.CPI.TOTL.ZG", "Inflation", "Inflation_CPI"),  # Inflación (IPC)
    ("SL.EMP.TOTL.SP.ZS", "Employment_Total_Pct", "Employment_Total"), # Empleo total (% de la población de 15+)
    ("SI.POV.GINI", "GINI_Index", "GINI_Coefficient"),       # Coeficiente GINI
    ("NV.IND.TOTL.ZS", "Industry_GDP_Pct", "Industry_GDP"),    # PIB del sector industrial (% del PIB)
    ("SP.POP.TOTL", "Total_Population", "Population_Total"),  # Población total
    ("NY.GDP.PCAP.CD", "GDP_per_Capita_USD", "GDP_per_Capita"),  # PIB per cápita (USD)
    ("NE.EXP.GNFS.ZS", "Exports_Goods_Services_Pct_GDP", "Exports_Pct_GDP"),  # Exportaciones (% del PIB)
    ("NE.IMP.GNFS.ZS", "Imports_Goods_Services_Pct_GDP", "Imports_Pct_GDP"),  # Importaciones (% del PIB)
    ("GC.TAX.TOTL.GD.ZS", "Tax_Revenue_Pct_GDP", "Tax_Revenue"),  # Ingresos fiscales (% del PIB)
    ("SH.XPD.CHEX.GD.ZS", "Health_Expenditure_Pct_GDP", "Health_Expenditure"),  # Gasto en salud (% del PIB)
    ("SE.XPD.TOTL.GD.ZS", "Education_Expenditure_Pct_GDP", "Education_Expenditure"),  # Gasto en educación (% del PIB)
    ("SL.UEM.TOTL.ZS", "Unemployment_Rate", "Unemployment"),  # Tasa de desempleo (%)
    ("BX.KLT.DINV.CD.WD", "FDI_Net_Inflows_USD", "FDI_Net_Inflows"),  # Inversión extranjera directa (USD)
    ("EG.USE.PCAP.KG.OE", "Energy_Use_per_Capita", "Energy_Use_Capita"),  # Consumo energético per cápita (kg de petróleo equivalente)
    ("DT.DOD.DECT.CD", "External_Debt_Stock_USD", "External_Debt"),  # Deuda externa total (USD)
    ("DT.DOD.DECT.GN.ZS", "External_Debt_Pct_GDP", "External_Debt_GDP")  # Deuda externa total (% del PIB)


]

# Rango de años para la consulta
START_YEAR = 1960
END_YEAR = 2024

# --- CAMBIO AQUÍ: Definir la ruta y el nombre del archivo Excel ---
# Por ejemplo, si quieres guardarlo en una carpeta llamada 'MisDatos' dentro de tu unidad C:
output_directory = r"C:\Users\asus\OneDrive\Data Scientist\Datasets" # Usa 'r' antes de la cadena para rutas de Windows para evitar problemas con '\'
output_filename = "World_Bank_Indicators_Extended.xlsx"

# Combinar la ruta del directorio con el nombre del archivo
EXCEL_FILE_NAME = os.path.join(output_directory, output_filename)
# ------------------------------------------------------------------

# Asegurarse de que el directorio de salida exista
os.makedirs(output_directory, exist_ok=True)


# Crear un objeto ExcelWriter para escribir en diferentes hojas
with pd.ExcelWriter(EXCEL_FILE_NAME, engine='xlsxwriter') as writer:
    for indicator_code, column_name, sheet_name in INDICATORS:
        print(f"Fetching data for indicator: {indicator_code}...")

        url = f"http://api.worldbank.org/v2/country/all/indicator/{indicator_code}"
        params = {
            "date": f"{START_YEAR}:{END_YEAR}",
            "format": "json",
            "per_page": "30000"
        }

        response = requests.get(url, params=params)
        data = response.json()

        if response.status_code == 200 and data and len(data) > 1:
            extracted_data = []
            for entry in data[1]:
                country = entry['country']['value']
                year = entry['date']
                value = entry['value']
                extracted_data.append([country, year, value])

            df = pd.DataFrame(extracted_data, columns=["Country", "Year", column_name])
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"Data for {indicator_code} saved to sheet '{sheet_name}'.")
        else:
            print(f"Could not fetch data for {indicator_code}. Response: {response.status_code}, Data: {data}")

print(f"\nAll data has been successfully saved to '{EXCEL_FILE_NAME}'.")

Fetching data for indicator: NY.GDP.MKTP.CD...
Data for NY.GDP.MKTP.CD saved to sheet 'GDP_Current_USD'.
Fetching data for indicator: SP.POP.1564.TO...
Data for SP.POP.1564.TO saved to sheet 'Population_15-64'.
Fetching data for indicator: FP.CPI.TOTL.ZG...
Data for FP.CPI.TOTL.ZG saved to sheet 'Inflation_CPI'.
Fetching data for indicator: SL.EMP.TOTL.SP.ZS...
Data for SL.EMP.TOTL.SP.ZS saved to sheet 'Employment_Total'.
Fetching data for indicator: SI.POV.GINI...
Data for SI.POV.GINI saved to sheet 'GINI_Coefficient'.
Fetching data for indicator: NV.IND.TOTL.ZS...
Data for NV.IND.TOTL.ZS saved to sheet 'Industry_GDP'.
Fetching data for indicator: SP.POP.TOTL...
Data for SP.POP.TOTL saved to sheet 'Population_Total'.
Fetching data for indicator: NY.GDP.PCAP.CD...
Data for NY.GDP.PCAP.CD saved to sheet 'GDP_per_Capita'.
Fetching data for indicator: NE.EXP.GNFS.ZS...
Data for NE.EXP.GNFS.ZS saved to sheet 'Exports_Pct_GDP'.
Fetching data for indicator: NE.IMP.GNFS.ZS...
Data for NE.IMP.